##### 저장 된 모델 활용
- 모델 파일 종류
    * 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 가능
    * 모델 전체 저장 파일 => 바로 로딩 후 사용 가능

[1] 모듈 로딩 <hr>

In [9]:
# 모듈로딩
# Model 관련
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.classification import F1Score, BinaryF1Score, BinaryConfusionMatrix
from torchinfo import summary

In [5]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = './model/best_model/'

# 파일명
SAVE_MODEL ='best_model4.pth'


In [6]:
class SentenceClassifier(nn.Module):
    def __init__(self,
                 n_vocab,
                 hidden_dim,
                 embedding_dim,
                 n_layers, 
                 dropout = 0.5,
                 bidirectional = True,
                 model_type = 'lstm'):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim = embedding_dim,
            padding_idx= 0)
        
        if model_type =='rnn':
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size = hidden_dim,
                num_layers= n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True)
    
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size = hidden_dim,
                num_layers= n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True )
            
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
            
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings =self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self. dropout(last_output)
        logits = self. classifier(last_output)
        return logits

[2] 모듈 로딩- 모델 전체 파일 사용

In [7]:
Model = torch.load(SAVE_PATH+SAVE_MODEL, weights_only=False)
# weights_only= ''>> 모델을 저장하거나 로드할 때 사용하는 파라미터
# True로 설정하면 모델의 가중치만 저장되고
# False로 설정하면 모델의 전체 상태 (구조, 옵티마이저 상태 등)가 함께 저장

In [10]:
summary(Model)

Layer (type:depth-idx)                   Param #
SentenceClassifier                       --
├─Embedding: 1-1                         1,280,256
├─LSTM: 1-2                              198,656
├─Linear: 1-3                            129
├─Dropout: 1-4                           --
Total params: 1,479,041
Trainable params: 1,479,041
Non-trainable params: 0

[3] 예측 <hr>

- 피쳐/라벨 : 
    + firstBlood : 킬 300골 + 추가 보너스 100골       
    + firstTower : 팀 전체 150골      
    + firstInhibitor : 억제기 >> 슈퍼 미니언 등장       
    + firstBaron : 바론 >> 공증     
    + firstDragon : 드래곤 >> 여러 버프      
    + firstRiftHerald : 남작 >> 타워 철거 도움 

In [12]:
lol=[float(x) for x in input('FB, FT, FI, FBaron, FD, FRH').split(',')] # 데이터 대입 
lol

[0.0, 1.0, 0.0, 1.0, 1.0, 1.0]

In [13]:
dataTS=torch.FloatTensor(lol).reshape(1,-1)
dataTS.shape, dataTS

(torch.Size([1, 6]), tensor([[0., 1., 0., 1., 1., 1.]]))

In [14]:
# 새로운 데이터에 대한 예측 즉, predict
LOLModel.eval() # 검증모드 켜기!
with torch.no_grad():
    # 추론/평가
    pre_val=LOLModel(dataTS)

print(pre_val)

tensor([[0.6651]])


In [9]:
def pridict():
    pass

* * *